In [1]:
from IS2_preprocessing import detect_level_ice
from IS2_preprocessing import read_ATL10

In [11]:
import xarray as xr
import h5py
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from scipy.interpolate import lagrange

In [48]:
file = '/glade/scratch/mollyw/external_data/ICESat-2/freeboard/2019.01.01/ATL10-01_20190101005132_00550201_005_02.h5'
track = 'gt1r'

In [50]:
def read_track(file, track):
    with h5py.File(file, mode='r') as f:
        latvar = f['/%s/freeboard_beam_segment/beam_freeboard/latitude' % track]
        latitude = latvar[:]
    
        # access the longitude
        lonvar = f['/%s/freeboard_beam_segment/beam_freeboard/longitude' % track]
        longitude = lonvar[:]

        # access the freeboard values 
        datavar = f['/%s/freeboard_beam_segment/beam_freeboard/beam_fb_height' % track]
        data = datavar[:]

        # access the freeboard uncertainty
        unc_var = f['/%s/freeboard_beam_segment/beam_freeboard/beam_fb_sigma' % track]
        unc = unc_var[:]

        # access the open ocean mask
        mask_var = f['/%s/freeboard_beam_segment/height_segments/height_segment_ssh_flag' % track]
        mask = mask_var[:]

        # access the surface type classification
        surf_var = f['/%s/freeboard_beam_segment/height_segments/height_segment_type' % track]
        surf = surf_var[:]

        #handle FillValue
        _FillValue = datavar.attrs['_FillValue']
        data[data == _FillValue] = np.nan
        unc[unc == _FillValue] = np.nan

        # collect time information
        timevar = f['/%s/freeboard_beam_segment/delta_time' % track]
        time = timevar[:]

        return data, latitude, longitude


In [20]:
def haversine_distance(x1, x2, y1, y2):

    dist = 2*np.arcsin(np.sqrt(np.sin((x1 - y1)/2)**2 +
                               np.cos(x1)*np.cos(y1)*np.sin((x2-y2)/2)**2))
    return dist

In [55]:
def latlon2dist(data, latitude, longitude):
        x_interp = []
        for i in range(0, len(data)-1):
                x1 = latitude[i]
                y1 = longitude[i]

                x2 = latitude[i+1]
                y2 = longitude[i+1]

                dist = 1000 * haversine_distance(x1, x2, y1, y2)
                x_interp.append(dist)

        distances = np.cumsum(x_interp)
        distances = np.insert(distances, 0, 0)

        return distances

In [ ]:
def interp_100m(data, latitude, longitude, distances):

    x_new = np.arange(0, distances[-1], 100)

    new_data = np.interp(x_new, distances, data)
    new_lats = np.interp(x_new, distances, latitude)
    new_lons = np.interp(x_new, distances, longitude)

    return new_data, new_lats, new_lons

In [ ]:
def create_mask(data, distances):

    
    # calculate the gradient of the data
    gradient = np.gradient(data)

    # create first mask 
    mask = []
    for x in range(0, len(gradient)):
        if np.isnan(gradient[x]):
            mask.append(np.nan)
        elif abs(gradient[x]) > 0.04:
            mask.append(0)
        else:
            mask.append(1)

    return mask 
    # 